# TDM 직접구현 

In [2]:
docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']


In [7]:
doc_ls = []
for doc in docs: 
    doc_ls.append(doc.split(' ')) # 공백으로 토큰화 해서  doc_ls에 할당
    
doc_ls


[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [8]:
from collections import defaultdict
word2id = defaultdict(lambda: len(word2id))
for doc in doc_ls:
    for token in doc:
        word2id[token] # 꺾쇠괄호는 인덱스의미
        
print(word2id)
        

defaultdict(<function <lambda> at 0x00000242AE8ABE18>, {'동물원': 0, '코끼리': 1, '원숭이': 2, '바나나': 3, '엄마': 4, '아기': 5})


In [18]:
import numpy as np

TDM = np.zeros((len(word2id), len(doc_ls)), dtype=int)
print(TDM)

for i, doc in enumerate(doc_ls):
    for token in doc:
        TDM[word2id[token],i ] +=1 # 해당 토큰의 위치(column)
    # 하나의 array로 만들면 되기 떄문에 따로 리스트 append 할 필요없음(bow에서 한거같이..할필요 ㅌ)
    # 행렬로 표기(BOW와 차이점: BOW는 1차원 배열 )
TDM

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


array([[1, 1, 0, 0],
       [1, 0, 2, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [22]:
import pandas as pd

doc_names = ['문서'+ str(i) for i in range(len(doc_ls))]
print('doc_names',doc_names)

doc_names ['문서0', '문서1', '문서2', '문서3']


In [24]:
sorted_vocab=sorted((value,key) for key, value in word2id.items())
vocab= [v[1] for v in sorted_vocab]
df_TDM = pd.DataFrame(TDM, columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
원숭이,0,1,0,1
바나나,0,1,0,2
엄마,0,0,1,0
아기,0,0,1,0


sklearn 패키지를 이용해서 TDM을 만들 수 있는데, 

countvectorizer는 토큰 갯수(자동으로 공백으로 토큰화해줌)를 세주는 패키지 , 
inpu할 doc를 fit_transform해주고
array 해줘서 배열해주기( 확인하고 )

sklearn은 자동적으로 DTM으로 뽑아주기 때문에
TDM을 뽑고자 한다면 T 를 붙여서 transform 해줘야 함!

TDM 을 데이터 프레임으로 만들어주면 됨 

토큰화 한걸 딕셔너리에 넣어서 
id2word는 유니크한 토큰이 몇개나 있는지 보여주는 애들 

그리고 실제에선 TDM 메트릭스만 만드는 것만 ! 


In [26]:
# sklearn을 사용해서 TDM

In [27]:
docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
# sklearn은 DTM으로 만들게 설정되어 있음
count_vect = CountVectorizer()
DTM=count_vect.fit_transform(docs)
print(DTM)

DTM.toarray()

  (0, 5)	1
  (0, 0)	1
  (1, 1)	1
  (1, 4)	1
  (1, 0)	1
  (2, 2)	1
  (2, 3)	1
  (2, 5)	2
  (3, 1)	2
  (3, 4)	1
  (3, 5)	1


array([[1, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 2],
       [0, 2, 0, 0, 1, 1]], dtype=int64)

In [35]:
DTM.toarray().T  # TDM 으로 trans

array([[1, 1, 0, 0],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 2, 1]], dtype=int64)

In [36]:
# 여기서부는 시각화 해주는 과정이라 위에까지만 하면 TDM 만든거 끝

In [45]:
import pandas as pd 

doc_names =['문서'+ str(i) for i in range(len(doc_ls))]
print(doc_names)
vocab= count_vect.get_feature_names()
print(vocab)
df_TDM= pd.DataFrame(DTM.toarray().T, columns=doc_names)
df_TDM
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

['문서0', '문서1', '문서2', '문서3']
['동물원', '바나나', '아기', '엄마', '원숭이', '코끼리']


,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
바나나,0,1,0,2
아기,0,0,1,0
엄마,0,0,1,0
원숭이,0,1,0,1
코끼리,1,0,2,1


# gensim

In [47]:
docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']


In [49]:
import gensim
from gensim import corpora

doc_ls = [doc.split() for doc in docs] # 공백으로 토큰화
id2word = corpora.Dictionary(doc_ls)
TDM= [id2word.doc2bow(doc) for doc in doc_ls]
TDM


[[(0, 1), (1, 1)],
 [(0, 1), (2, 1), (3, 1)],
 [(1, 2), (4, 1), (5, 1)],
 [(1, 1), (2, 2), (3, 1)]]

In [50]:
# 여기서 부턴 시각화

In [53]:
from gensim.matutils import sparse2full
import pandas as pd
import numpy as np
doc_names=['문서'+str(i) for i in range(len(doc_ls))]
doc_names 
vocab= [id2word[i] for i in id2word.keys()]
DTM_matrix = [sparse2full(doc, len(vocab)).tolist() for doc in TDM]

df_TDM = pd.DataFrame(np.array(DTM_matrix, dtype=int).T, 
                     columns=doc_names)

df_TDM['단어']= vocab

df_TDM.set_index('단어')

,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
바나나,0,1,0,2
원숭이,0,1,0,1
아기,0,0,1,0
엄마,0,0,1,0


gensim이라는 라이브러리안에 corpora라는 패키지가 있는 것이에요. corpora라는 패키지는 단어 뭉치 사전 이라고 생각사시면되요. 그래서 corpora라는 패키지 안에 있는 dictioanary를 사용해서 unique한 단어를 뽑아줍니다.